# Experiment 4
- Load the saved model and replace the output layer of the model, as well as the two last convolutional layers.
- Train and evaluate the model on the cats and dogs dataset.

## Steps
- Load the saved model using keras.models.load_model().
- Freeze all layers except the output layer and the last two convolutional layers.
- Replace the output layer to match the number of classes (num_classes = 2 for cats vs. dogs).
- Retrain the model on the Cats vs. Dogs dataset.

### Load the saved model

In [14]:
import tensorflow as tf
from tensorflow import data as tf_data
from tensorflow import keras
from keras import layers

# Load the pre-trained model
saved_model_path = "models/experiment1_model.keras"
model = keras.models.load_model(saved_model_path)

### Modify the model
See which layer need to be reset, in our case 23 to 30

In [15]:
i = 0
for layer in model.layers:
    print(f"Layer {i}: {layer.name}")
    i += 1

Layer 0: input_layer_1
Layer 1: rescaling_1
Layer 2: conv2d_4
Layer 3: batch_normalization_8
Layer 4: activation_8
Layer 5: activation_9
Layer 6: separable_conv2d_7
Layer 7: batch_normalization_9
Layer 8: activation_10
Layer 9: separable_conv2d_8
Layer 10: batch_normalization_10
Layer 11: max_pooling2d_3
Layer 12: conv2d_5
Layer 13: add_3
Layer 14: activation_11
Layer 15: separable_conv2d_9
Layer 16: batch_normalization_11
Layer 17: activation_12
Layer 18: separable_conv2d_10
Layer 19: batch_normalization_12
Layer 20: max_pooling2d_4
Layer 21: conv2d_6
Layer 22: add_4
Layer 23: activation_13
Layer 24: separable_conv2d_11
Layer 25: batch_normalization_13
Layer 26: activation_14
Layer 27: separable_conv2d_12
Layer 28: batch_normalization_14
Layer 29: max_pooling2d_5
Layer 30: conv2d_7
Layer 31: add_5
Layer 32: separable_conv2d_13
Layer 33: batch_normalization_15
Layer 34: activation_15
Layer 35: global_average_pooling2d_1
Layer 36: dropout_1
Layer 37: dense_1


In [16]:
def make_model(input_shape, num_classes):
    inputs = keras.Input(shape=input_shape)

    # Entry block
    x = layers.Rescaling(1.0 / 255)(inputs)
    x = layers.Conv2D(128, 3, strides=2, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    previous_block_activation = x  # Set aside residual

    for size in [256, 512, 728]:
        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(size, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(size, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.MaxPooling2D(3, strides=2, padding="same")(x)

        # Project residual
        residual = layers.Conv2D(size, 1, strides=2, padding="same")(
            previous_block_activation
        )
        x = layers.add([x, residual])  # Add back residual
        previous_block_activation = x  # Set aside next residual

    x = layers.SeparableConv2D(1024, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    x = layers.GlobalAveragePooling2D()(x)
    if num_classes == 2:
        units = 1
    else:
        units = num_classes

    x = layers.Dropout(0.25)(x)
    # Change output layer for multi-class classification (120 dog breeds)
    outputs = layers.Dense(units, activation="softmax")(x)
    return keras.Model(inputs, outputs)

In [17]:
# Create a fresh model with the same architecture
fresh_model = make_model(input_shape=(180, 180) + (3,), num_classes=1)
base_model = keras.Model(inputs=model.input, outputs=model.layers[-2].output)
base_model.trainable = False 

# Reset weights for layers 6 to 22
for i in range(14, 31):
    base_model.layers[i].set_weights(fresh_model.layers[i].get_weights())
    base_model.layers[i].trainable = True  # Make sure the layers are trainable

In [18]:
# Add new output layer (Binary Classification)
new_output = layers.Dense(1, activation="sigmoid")(base_model.output)

# Create a new model
new_model = keras.Model(inputs=base_model.input, outputs=new_output)

In [19]:
# Ensure the new model is compiled with the goof layer unfrozen
for i, layer in enumerate(new_model.layers):
    print(f"{i}: {layer.name} — Trainable: {layer.trainable}")

0: input_layer_1 — Trainable: False
1: rescaling_1 — Trainable: False
2: conv2d_4 — Trainable: False
3: batch_normalization_8 — Trainable: False
4: activation_8 — Trainable: False
5: activation_9 — Trainable: False
6: separable_conv2d_7 — Trainable: False
7: batch_normalization_9 — Trainable: False
8: activation_10 — Trainable: False
9: separable_conv2d_8 — Trainable: False
10: batch_normalization_10 — Trainable: False
11: max_pooling2d_3 — Trainable: False
12: conv2d_5 — Trainable: False
13: add_3 — Trainable: False
14: activation_11 — Trainable: True
15: separable_conv2d_9 — Trainable: True
16: batch_normalization_11 — Trainable: True
17: activation_12 — Trainable: True
18: separable_conv2d_10 — Trainable: True
19: batch_normalization_12 — Trainable: True
20: max_pooling2d_4 — Trainable: True
21: conv2d_6 — Trainable: True
22: add_4 — Trainable: True
23: activation_13 — Trainable: True
24: separable_conv2d_11 — Trainable: True
25: batch_normalization_13 — Trainable: True
26: activati

### Compile the model

In [20]:
new_model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.0001), # Modification for experiment 1
    loss=keras.losses.BinaryCrossentropy(),
    metrics=[keras.metrics.BinaryAccuracy(name="acc")],
)

### Train and evaluate the model

In [21]:
# Create the dataset
image_size = (180, 180)
batch_size = 128

train_ds, val_ds = keras.utils.image_dataset_from_directory(
    "PetImages",
    validation_split=0.2,
    subset="both",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
)

data_augmentation_layers = [
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
]


def data_augmentation(images):
    for layer in data_augmentation_layers:
        images = layer(images)
    return images

augmented_train_ds = train_ds.map(
    lambda x, y: (data_augmentation(x), y))

# Apply `data_augmentation` to the training images.
train_ds = train_ds.map(
    lambda img, label: (data_augmentation(img), label),
    num_parallel_calls=tf_data.AUTOTUNE,
)
# Prefetching samples in GPU memory helps maximize GPU utilization.
train_ds = train_ds.prefetch(tf_data.AUTOTUNE)
val_ds = val_ds.prefetch(tf_data.AUTOTUNE)

Found 23422 files belonging to 2 classes.
Using 18738 files for training.
Using 4684 files for validation.


In [22]:
epochs = 50
callbacks = [
    keras.callbacks.ModelCheckpoint(filepath="models/experiment4_epoch_{epoch}.keras")
]

new_model.fit(
    train_ds,
    epochs=epochs,
    validation_data=val_ds,
    callbacks=callbacks
)

new_model.save("experiment4_model.keras")


Epoch 1/50
147/147 ━━━━━━━━━━━━━━━━━━━━ 928s 5s/step - acc: 0.6208 - loss: 0.6621 - val_acc: 0.4966 - val_loss: 0.7056
Epoch 2/50
147/147 ━━━━━━━━━━━━━━━━━━━━ 560s 4s/step - acc: 0.7243 - loss: 0.5447 - val_acc: 0.4979 - val_loss: 0.7116
Epoch 3/50
147/147 ━━━━━━━━━━━━━━━━━━━━ 562s 4s/step - acc: 0.7513 - loss: 0.5105 - val_acc: 0.5096 - val_loss: 0.7054
Epoch 4/50
147/147 ━━━━━━━━━━━━━━━━━━━━ 562s 4s/step - acc: 0.7797 - loss: 0.4654 - val_acc: 0.5807 - val_loss: 0.6693
Epoch 5/50
147/147 ━━━━━━━━━━━━━━━━━━━━ 558s 4s/step - acc: 0.8020 - loss: 0.4300 - val_acc: 0.6552 - val_loss: 0.6029
Epoch 6/50
147/147 ━━━━━━━━━━━━━━━━━━━━ 558s 4s/step - acc: 0.8212 - loss: 0.3967 - val_acc: 0.8081 - val_loss: 0.4239
Epoch 7/50
147/147 ━━━━━━━━━━━━━━━━━━━━ 558s 4s/step - acc: 0.8329 - loss: 0.3743 - val_acc: 0.8456 - val_loss: 0.3604
Epoch 8/50
147/147 ━━━━━━━━━━━━━━━━━━━━ 558s 4s/step - acc: 0.8485 - loss: 0.3439 - val_acc: 0.8427 - val_loss: 0.3593
Epoch 9/50
147/147 ━━━━━━━━━━━━━━━━━━━━ 558s 4s/